## Item 37: Compose Classes Instead of Nesting Many Levels of Built-in Type

1. Avoid making dictionaries with values that are dictionaries, long tuples, or complex nesting of other buit-in types.
2. Use named tuple for lightweight, immutable data containers before you need the flexibility of a full class. 
3. Using multiple classes when your Internal state dictionaries get complicated.

In [3]:
from collections import namedtuple, defaultdict


Grade = namedtuple('Grade', ('score', 'weight')) #class name = Grade이고, score와 weight를 feature로 가짐

class Subject:
    def __init__(self):
        self._grades = []
    
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight)) #named tuple 사용
    
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject) #class로 해도 된다
    
    def get_subject(self, name):
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)

    def get_student(self, name):
        return self._students[name]
    

In [4]:
book = Gradebook()
albert = book.get_student('Albert Einstein')
math = albert.get_subject('Math')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
gym = albert.get_subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade())

80.25


## Item 38: Accept Functions Instead of Classes for Simple Interfaces

In [6]:
def log_missing():
    print('Key added')
    return 0

current = {'green': 12, 'blue': 3}
increments = [('red', 5), ('blue', 17), ('orange', 9)]

result = defaultdict(log_missing, current) #default값은 log_missing function
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


In [10]:
class BetterCountMissing:
    def __init__(self):
        self.added = 0
    
    def __call__(self):
        self.added += 1
        return 0

counter =BetterCountMissing()
result = defaultdict(counter, current)  # __call__ 함수
for key, amount in increments:
    result[key] += amount

print(counter.added)

2


## Item 39: Use @classmethod Polymorphism to Coustruct Object Generically



In [11]:
class GenericInputData:
    def read(self):
        raise NotImplementedError
    
    @classmethod
    def generate_inputs(cls, config): #cls 변수를 통해 class 접근
        raise NotImplementedError

## Item 40: Initialize Parent Classes with super

In [14]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class TimeSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7

class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9

class Goodway(TimeSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)

foo = Goodway(5)
print(foo.value) # 7*(5+9)

98


## Item 41: Consider Composing Functionality with Mix-in classes

## Item 42: Prefer Public Attributes Over Private Ones

1. Private attributes aren't rigorously enforced by the Python compiler
2. Only consider using private attributes to avoid naming conflicts with subclasses that are out of your control

In [15]:
class ApiClass:
    def __init__(self):
        self._value = 5 #private attribute로 할당
    
    def get(self):
        return self._value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'

a = Child()
print(f'{a.get()} and {a._value} should be different') #Conflict 발생

hello and hello should be different


In [16]:
class ApiClass:
    def __init__(self):
        self.__value = 5 #private attribute로 할당
    
    def get(self):
        return self.__value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'

a = Child()
print(f'{a.get()} and {a._value} should be different')

5 and hello should be different


## Item 43: Inherit from collections.abc for Custom Container Types

1. Beware of the large number of methods required to implement custom container types correctly.
2. Have your custom container types inherit from the interfaces defined in collections.abc to ensure that your classes match required interfaces and behaviors.

In [17]:
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    
    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print('Length is', len(foo))
foo.pop()
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

Length is 7
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}
